In [1]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
import cv2
import time
from scipy.io import savemat

In [2]:
print(sns.__version__)

0.11.1


histograms in seaborn here: https://www.machinelearningplus.com/plots/matplotlib-histogram-python-examples/

In [3]:
save_location =  "../FLIR_Camera"
pdfCombined_figs_location  = "../FLIR_Camera/fastFlow_ShallowH_test1_centeredCame_flume_LookAngle_35Deg_PDFCOMBINED_FIGURES"

In [4]:
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
colors = {'corals':'dodgerblue','canopy':'green', 'rocks':'orange', 'dunes':'brown'}
bed_data = {}
bed = BEDFORMS[3]
for BEDFORM in [bed]: #  manually change this bedform everywhere
    data_filename = BEDFORM + "_fastFlow_ShallowH_test1.pickle"
    with open(os.path.join(save_location,data_filename), 'rb') as handle:
        all_data_dict_loaded = pickle.load(handle)
    bed_data[BEDFORM] = all_data_dict_loaded

## Save the 3D FFT (for x,y,t) to visualize using Slicing in Matlab

In [5]:
for BEDFORM in [bed]:    
    
    s_hat_x = np.fft.fftn(bed_data[BEDFORM]['all_sx_maps'], axes = ([0,1,2]))
    s_hat_x = np.fft.fftshift(s_hat_x)
    s_hat_x_mag = (np.log(np.abs(s_hat_x)))**2

    s_hat_y = np.fft.fftn(bed_data[BEDFORM]['all_sy_maps'], axes = ([0,1,2]))
    s_hat_y = np.fft.fftshift(s_hat_y) # DC content at the center
    s_hat_y_mag = (np.log(np.abs(s_hat_y)))**2
    

In [6]:
M,N,K = s_hat_x_mag.shape
lim = 80 # total window aroun 0 frequency    
s_hat_x_mag = s_hat_x_mag[M//2 - lim//2:M//2 + lim//2, N//2 - lim//2:N//2 + lim//2,:] 
s_hat_y_mag = s_hat_y_mag[M//2 - lim//2:M//2 + lim//2, N//2 - lim//2:N//2 + lim//2,:]    

In [7]:
print(s_hat_x_mag.shape)

(80, 80, 73)


In [8]:
FreqCompRows = np.fft.fftfreq(s_hat_x.shape[0],d=1)
FreqCompCols = np.fft.fftfreq(s_hat_x.shape[1],d=1)
FreqCompVert = np.fft.fftfreq(s_hat_x.shape[2],d=1)
FreqCompRows = np.fft.fftshift(FreqCompRows)[M//2 - lim//2:M//2 + lim//2]
FreqCompCols = np.fft.fftshift(FreqCompCols)[N//2 - lim//2:N//2 + lim//2]
FreqCompVert = np.fft.fftshift(FreqCompVert)

R,C,V = np.meshgrid(FreqCompRows, FreqCompCols, FreqCompVert)

In [9]:
print(s_hat_x_mag.shape, FreqCompRows.shape, FreqCompCols.shape, FreqCompVert.shape)

(80, 80, 73) (80,) (80,) (73,)


In [10]:
dic = {"R":R,"C":C,"V":V \
         ,"FreqCompRows":FreqCompRows, "FreqCompCols":FreqCompCols, "FreqCompVert":FreqCompVert\
        ,"s_hat_x_mag":s_hat_x_mag, "s_hat_y_mag":s_hat_y_mag\
        ,'BEDFORM':'dunes'}
savemat('../FLIR_Camera/3DFFT_{}.mat'.format(bed),dic)